# 🚀 GPU vs CPU Benchmarking for Insect Classification
This notebook compares training performance (speed and accuracy) of a fine-tuned MobileNetV3Large model on **GPU vs CPU** using TensorFlow.

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Setup directories (update paths if needed)
train_path = 'dataset/train'
val_path = 'dataset/validation'

# Image generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_path, target_size=(224, 224),
                                              batch_size=32, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_path, target_size=(224, 224),
                                          batch_size=32, class_mode='categorical')


In [ ]:
def create_model(num_classes):
    base_model = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg')
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
with tf.device('/CPU:0'):
    print("Training on CPU...")
    model_cpu = create_model(train_gen.num_classes)
    start_time = time.time()
    history_cpu = model_cpu.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[EarlyStopping(patience=3)])
    cpu_time = time.time() - start_time
    print(f"CPU training time: {cpu_time:.2f} seconds")


In [ ]:
if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        print("Training on GPU...")
        model_gpu = create_model(train_gen.num_classes)
        start_time = time.time()
        history_gpu = model_gpu.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[EarlyStopping(patience=3)])
        gpu_time = time.time() - start_time
        print(f"GPU training time: {gpu_time:.2f} seconds")
else:
    print("No GPU found. Please enable GPU runtime in Colab.")


In [ ]:
try:
    print(f"Summary:\nCPU training time: {cpu_time:.2f} sec\nGPU training time: {gpu_time:.2f} sec")
except:
    print("Only CPU test was completed.")
